## Two Pipe Lines - Modelling

#### José María Benítez, Pablo Berástegui, Daniel Escobosa y Alejandro de Haro

### Importing packages and modules

In [10]:
import pyomo.environ as pe
import pyomo.opt as po

### Create the model

In [11]:
model = pe.ConcreteModel()

$ \mathcal{N} $: network nodes ${A,B,C,D,E,F,G,P1,P2}$

$ \mathcal{R} $: refineries ${A,B,C,D,E,F,G}$

$ \mathcal{P} $: pipelines ${P1,P2}$

$ \mathcal{E}_{RR} $: arcs (connections) between refineries $\subseteq \mathcal{R}\times\mathcal{R}$

$ \mathcal{E}_{PR} $: possible pipeline–refinery pairs $=\mathcal{P}\times\mathcal{R}$

$ \mathcal{N}(r) $: neighbors of refinery $r$ in $\mathcal{E}_{RR}$

In [12]:
model.N  = pe.Set(initialize=['A','B','C','D','E','F','G','P1','P2'])
model.R  = pe.Set(initialize=['A','B','C','D','E','F','G'])
model.P  = pe.Set(initialize=['P1','P2'])

base_caps = {
    ('A','B'):600, ('B','C'):700, ('C','D'):700, ('D','E'):650,
    ('E','F'):550, ('F','A'):700, ('A','G'):850, ('B','G'):950,
    ('C','G'):600, ('D','G'):550, ('E','G'):700, ('F','G'):650
}

RR = []
for i,j in base_caps:
    RR.append((i,j))
    RR.append((j,i))
model.E_RR = pe.Set(dimen=2, initialize=RR)

#### Parameters

$ Q_{ij} $: available capacity on arc $(i,j)\in\mathcal{E}_{RR}$ $[\ell/s]$

$ D_{r} $: contracted demand at refinery $r\in\mathcal{R}$ $[\ell/s]$

$ PD_{r} $: penalty of unmet demand at refinery $r$ $[/€(\ell/s)]$

$ S_{p} $: supply capacity of pipeline $p\in\mathcal{P}$ $[\ell/s]$

In [13]:
def Q_init(m,i,j):
    if (i,j) in base_caps:
        return base_caps[(i,j)]
    return base_caps[(j,i)]
model.Q = pe.Param(model.E_RR, initialize=Q_init, within=pe.NonNegativeReals)
model.E_PR = pe.Set(dimen=2, initialize=[(p,r) for p in model.P for r in model.R])

D_data  = {r:1200 for r in model.R}
PD_data = {r:1.0  for r in model.R}
S_data  = {'P1':1500,'P2':1500}

model.D  = pe.Param(model.R, initialize=D_data, within=pe.NonNegativeReals)
model.PD = pe.Param(model.R, initialize=PD_data, within=pe.NonNegativeReals)
model.S  = pe.Param(model.P, initialize=S_data,  within=pe.NonNegativeReals)

#### Variables

$ x_{ij}\ge 0 $: flow from refinery $i$ to $j$, $(i,j)\in\mathcal{E}_{RR}$ $[\ell/s]$

$ x_{pr}\ge 0 $: flow from pipeline $p$ to refinery $r$, $(p,r)\in\mathcal{E}_{PR}$ $[\ell/s]$

$ nsd_{r}\ge 0 $: non-served demand at refinery $r\in\mathcal{R}$ $[\ell/s]$

$ u_{pr}\in{0,1} $: $1$ if pipeline $p$ connects to refinery $r$, $(p,r)\in\mathcal{E}_{PR}$

In [14]:
model.x   = pe.Var(model.E_RR, domain=pe.NonNegativeReals)
model.xpr = pe.Var(model.E_PR, domain=pe.NonNegativeReals)
model.nsd = pe.Var(model.R,    domain=pe.NonNegativeReals)
model.u   = pe.Var(model.E_PR, domain=pe.Binary)

#### Objective Function

$ \min Z = \sum_{r\in\mathcal{R}} PD_{r}; nsd_{r} $

In [15]:
def obj_rule(m):
    return sum(m.PD[r]*m.nsd[r] for r in m.R)
model.OBJ = pe.Objective(rule=obj_rule, sense=pe.minimize)

#### Constraints

Balance in refinery $\forall r\in\mathcal{R}$:
$ \sum_{(i,r)\in\mathcal{E}{RR}} x{ir} + \sum_{p\in\mathcal{P}} x_{pr} - \sum_{(r,j)\in\mathcal{E}{RR}} x{rj} = D_r - nsd_r $

Arc capacity $\forall (i,j)\in\mathcal{E}{RR}$:
$ x{ij} \le Q_{ij} $

Pipeline supply capacity $\forall (p,r)\in\mathcal{E}{PR}$:
$ x{pr} \le S_p , u_{pr} $

Each pipeline connects to at most one refinery $\forall p\in\mathcal{P}$:
$ \sum_{r\in\mathcal{R}} u_{pr} \le 1 $

No two pipelines can connect to adjacent refineries $\forall r\in\mathcal{R},;\forall p\neq p'\in\mathcal{P}$:
$ u_{pr} + \sum_{r'\in\mathcal{N}(r)} u_{p'r'} \le 1 $

Non-served demand bounded $\forall r\in\mathcal{R}$:
$ nsd_r \le D_r $

Domain restrictions:
$ x_{ij}\ge 0,; x_{pr}\ge 0,; nsd_r\ge 0,; u_{pr}\in{0,1} $

In [16]:
def cap_rr_rule(m,i,j):
    return m.x[i,j] <= m.Q[i,j]
model.Cap_RR = pe.Constraint(model.E_RR, rule=cap_rr_rule)

def cap_pr_rule(m,p,r):
    return m.xpr[p,r] <= m.S[p]*m.u[p,r]
model.Cap_PR = pe.Constraint(model.E_PR, rule=cap_pr_rule)

def bal_rule(m,r):
    inflow_rr = sum(m.x[i,r] for (i,rr) in m.E_RR if rr==r)
    inflow_p  = sum(m.xpr[p,r] for p in m.P)
    out_rr    = sum(m.x[r,j] for (rr,j) in m.E_RR if rr==r)
    return inflow_rr + inflow_p - out_rr == m.D[r] - m.nsd[r]
model.Balance = pe.Constraint(model.R, rule=bal_rule)

def single_conn_rule(m,p):
    return sum(m.u[p,r] for r in m.R) <= 1
model.SingleConn = pe.Constraint(model.P, rule=single_conn_rule)

def no_adjacent_rule(m,p,r):
    return pe.inequality(0, m.u[p,r] + sum(m.u[pp,rp] for pp in m.P if pp!=p for (ra,rb) in m.E_RR for rp in [rb] if ra==r), 1)
model.NoAdjacent = pe.Constraint(model.P, model.R, rule=no_adjacent_rule)

def max_nsd_rule(m,r):
    return m.nsd[r] <= m.D[r]
model.MaxNSD = pe.Constraint(model.R, rule=max_nsd_rule)

#### Solver definition and solve statement

In [17]:
solver = po.SolverFactory('gurobi')
solver.solve(model, tee=False)

{'Problem': [{'Name': 'x1', 'Lower bound': 5400.0, 'Upper bound': 5400.0, 'Number of objectives': 1, 'Number of constraints': 82, 'Number of variables': 59, 'Number of binary variables': 14, 'Number of integer variables': 14, 'Number of continuous variables': 45, 'Number of nonzeros': 266, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.006000041961669922', 'Error rc': 0, 'Time': 0.24863815307617188}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

#### Results

In [18]:
print('OBJ =', pe.value(model.OBJ))

print('\nFlujos entre refinerías (x[i,j])')
for i,j in model.E_RR:
    v = model.x[i,j].value
    if v and v>1e-6:
        print(i,'->',j,':',v)

print('\nFlujos de oleoducto a refinería (xpr[p,r])')
for p,r in model.E_PR:
    v = model.xpr[p,r].value
    if v and v>1e-6:
        print(p,'->',r,':',v)

print('\nConexiones activas de oleoductos (u[p,r])')
for p in model.P:
    for r in model.R:
        if model.u[p,r].value > 0.5:
            print(p,'->',r)

print('\nDemanda no servida por refinería (nsd[r])')
for r in model.R:
    print(r,':',model.nsd[r].value)


OBJ = 5400.0

Flujos entre refinerías (x[i,j])
B -> A : 350.0
B -> C : 700.0
C -> B : 100.0
C -> D : 700.0
E -> D : 550.0
E -> F : 550.0
A -> F : 700.0
G -> A : 350.0
G -> B : 950.0
G -> C : 600.0
D -> G : 550.0
E -> G : 700.0
F -> G : 650.0

Flujos de oleoducto a refinería (xpr[p,r])
P1 -> E : 1500.0
P2 -> E : 1500.0

Conexiones activas de oleoductos (u[p,r])
P1 -> E
P2 -> E

Demanda no servida por refinería (nsd[r])
A : 1200.0
B : 1200.0
C : 700.0
D : 500.0
E : 0.0
F : 600.0
G : 1200.0
